In [29]:
import pandas as pd
from pathlib import Path
import glob
import datetime
import re
import os
import numpy as np
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from datetime import date
import math


In [30]:
# home and time
home = Path.home()
todaystr = date.today().strftime('%Y-%m-%d')

# set up concat directories
targetFolder = Path(home, 'HP Inc','GPSTW SOP - 2021 日新','Project team','Upload folder ( for buyer update )')
FD_folder = Path(targetFolder, "FD_today","*.xlsx")
shortage_folder = Path(targetFolder ,"shortage_today","*.xlsx")
PNbaseDetail_folder = Path(targetFolder ,"PNbasedDetail_today","*.xlsx")

In [31]:
# concat FD
FD_files = []
for FD_file in glob.glob(str(Path(FD_folder))):
    FD = pd.read_excel(FD_file)
    FD_files.append(FD)
FD_all = pd.concat(FD_files)


In [32]:
# concat shortage
shortage_files = []
for shortage_file in glob.glob(str(Path(shortage_folder))):
    shortage = pd.read_excel(shortage_file)
    shortage_files.append(shortage)
Shortage_all = pd.concat(shortage_files)

In [33]:
# concat PNbaseDetail
PNbasedDetail_files = []
for PNbasedDetail_file in glob.glob(str(Path(PNbaseDetail_folder))):
    PNbasedDetail = pd.read_excel(PNbasedDetail_file)
    PNbasedDetail_files.append(PNbasedDetail)
PNbaseDetail_all = pd.concat(PNbasedDetail_files)

In [34]:
Shortage_all['len_FV'] = Shortage_all['FV'].str.len()
Shortage_all['len_platform'] = Shortage_all['Platform'].str.len()
Shortage_all = Shortage_all.sort_values(['len_FV','len_platform'] , ascending=[False,False])
Shortage_all.reset_index()


,index,ODM,Item,Commodity,FV,Platform,P1,Net P2,Net P3,Total Shortage Qty,BT shortage,Working on upside,ReportDate,last FD date,len_FV,len_platform
0,0,CEI,ADA67,AC ADAPTOR,90W TUBE Doris II,Frozone 22C1/Frozone 21C2,1817,0,0,1817,0,0,2022-11-01,2022-11-03 00:00:00,17,25
1,0,CQIEC,1.0,AC ADAPTOR,120W Delphin,AdicorA/AdicorD,1222181,0,0,1222181,0,0,2022-11-01,2022-12-19 00:00:00,12,15
2,1,CQIEC,5.0,AC ADAPTOR,65W Taroko tube USBC,ULTRONS,25,0,0,25,0,0,2022-11-01,2022-11-07 00:00:00,20,7
3,0,CQIEC,2.0,AC ADAPTOR,120W Delphin 7.4mm,HOOK2,60460,0,0,60460,0,0,2022-11-01,2022-11-17 00:00:00,18,5
4,1,CQIEC,3.0,AC ADAPTOR,180W Hydra,Nantucket,824,0,0,824,0,0,2022-11-01,2022-11-07 00:00:00,10,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,1,CQIEC,2.0,WEBCAM,1F NMc NKBL UB2 720HD MPG - 0FHU,Bran/Sansa,136,0,0,136,0,0,2022-11-01,2022-11-01 00:00:00,32,10
265,0,CQQCI,1.0,WEBCAM,1F DMc UB2 720HD -YSCE,B52 1C20/Aires/Lepus/B52 21C2,815,0,0,815,0,0,2022-11-01,2022-11-01 00:00:00,22,29
266,1,CQQCI,2.0,WEBCAM,1F NMc NKBL UB2 5MP MPG - 0F5U,Aryastark/Raza/Splendor,14811,0,0,14811,0,0,2022-11-01,2022-11-01 00:00:00,30,23
267,2,CQQCI,3.0,WEBCAM,2F NMc UB2 4MP IR - 2D4RU,Splendor,32,0,0,32,0,0,2022-11-01,2022-11-01 00:00:00,25,8


In [35]:
FD_all['len_FV'] = FD_all['FV'].str.len()
FD_all['len_platform'] = FD_all['Platform'].str.len()
FD_all = FD_all.sort_values(['len_FV','len_platform'] , ascending=[False,False])
FD_all.reset_index()

,ODM,Item,Commodity,FV,Platform,Supplier,HP PN,ReportDate,FDdate,FDQty
5,CQQCI,14,CPU,INTEL TIGERLAKE-UP3 I7-1165G7 NON-IPU,Able/Dezik/Strelka/Rainer - 20\nC2 (TGL)/Bibur...,Intel,M18965-102,2022-11-01,2022-11-12,8072
1,CQIEC,6.0,POWERCORDS,PWC C5 1.8m stkr Conventional JPN,Adicor A,LONGWELL,349756-101,2022-11-01,2022-11-30,716
0,CQIEC,1.0,AC ADAPTOR,120W Delphin,AdicorA/AdicorD,LITEON,L41423-001,2022-11-01,2022-11-02,23760
2,CQIEC,1.0,AC ADAPTOR,120W Delphin,AdicorA/AdicorD,LITEON,L41423-001,2022-11-01,2022-11-10,18000
3,CQIEC,1.0,AC ADAPTOR,120W Delphin,AdicorA/AdicorD,CHICONY,L41423-002,2022-11-01,2022-11-10,28080
...,...,...,...,...,...,...,...,...,...,...
0,FLH,1,HDD,1TB 7200 SATA ENT 3.5,unknown platform,NaN,L66482-001,2022-11-01,2022-11-11,20
0,FLH,3.0,AC ADAPTOR,230W Medusa RPOS,unknown platform,CHICONY,L76393-002,2022-11-01,2022-11-16,1632
1,FLH,3.0,AC ADAPTOR,230W Medusa RPOS,unknown platform,CHICONY,L76393-002,2022-11-01,2022-11-17,3314
1,FLH,1,HDD,8TB 7200 SATA ENT 3.5,unknown platform,NaN,M93945-001,2022-11-01,2022-11-11,20


In [36]:
PNbaseDetail_all = PNbaseDetail_all.sort_values(['GPS Remark','ODM use column1','ODM use column2','ODM use column3','ODM use column4','ODM use column5'], ascending=[True,True,True,True,True,True])
PNbaseDetail_all.head(10)

FD_all['len_GPS Remark'] = FD_all['GPS Remark'].str.len()
FD_all['len_ODM use column1'] = FD_all['ODM use column1'].str.len()
FD_all['len_ODM use column2'] = FD_all['ODM use column2'].str.len()
FD_all['len_ODM use column3'] = FD_all['ODM use column3'].str.len()
FD_all['len_ODM use column4'] = FD_all['ODM use column4'].str.len()
FD_all['len_ODM use column5'] = FD_all['ODM use column5'].str.len()
FD_all = FD_all.sort_values(['len_GPS Remark','len_ODM use column1','len_ODM use column2','len_ODM use column3','len_ODM use column4','len_ODM use column5'] ,ascending=[True,True,True,True,True,True])
FD_all.reset_index()

,Item,HP PN,GPS Remark,852 stock,852 stock change,Over pull qty,ODM use column1,ODM use column2,ODM use column3,ODM use column4,ODM use column5,ReportDate
0,27.0,L31528-002,*Can only support WD L31528-002 on Space X and...,0,0,0,0GCA/0GCB/F93/0GN,Blondie/Space X/Lepus,NaN,NaN,NaN,2022-11-01
0,17.0,M54701-402,"CIS order without forecast, L/T 8-12 WKs",2592,0,0,CIS Order,NaN,NaN,NaN,NaN,2022-11-01
0,1,L50352-001,L50352-001 is EOL. ECN to replace with M10394-...,0,300,0,single source,NaN,NaN,NaN,NaN,2022-11-01
0,1,L50355-001,"L50355-001 is EOL without supply, waiting for ...",0,0,0,"CIS order use L50355-001 only, waiting for CIS...",NaN,NaN,NaN,NaN,2022-11-01
0,2,L70939-001,L70939-001 is EOL. Need CIS team to do EM to M...,0,0,0,"CIS order use L70939-001 only, waiting for CIS...",NaN,NaN,NaN,NaN,2022-11-01
0,19.0,M73974-401,"No FCST of this FV, L/T 8-12WKs.",0,0,0,CIS Order,NaN,NaN,NaN,NaN,2022-11-01
1,18.0,L76248-401,"No FCST of this FV, L/T 8-12WKs.",0,0,0,CIS Order,NaN,NaN,NaN,NaN,2022-11-01
1,2.0,L97621-106,SAB cut off 11/2,40,0,0,SAB cut off 11/2,upside,NaN,NaN,NaN,2022-11-01
0,1.0,L97615-105,SAB cut off 11/2,20,0,0,SAB cut off 11/2,NaN,NaN,NaN,NaN,2022-11-01
1,3.0,L56543-004,SG: CQIEC feedbacked SG on 10/28 as other mate...,690,720,0,NaN,NaN,NaN,NaN,NaN,2022-11-01


In [37]:
# apache airflow to upload SQL ( currently to desktop )
FD_all.to_excel(Path(home, 'Desktop', 'FD_all.xlsx'), index=False)
Shortage_all.to_excel(Path(home, 'Desktop', 'Shortage_all.xlsx'), index=False)
PNbaseDetail_all.to_excel(Path(home, 'Desktop', 'PNbaseDetail_all.xlsx'), index=False)